In [95]:
import os
import pandas as pd
import unicodedata
import re
import numpy as np
from docx import Document


In [96]:
regex="([x][a])\w+"
reg2='[\\xa0]\w+'
reg3='[\\xa0]'
reg4='[\\u200f]'
shofet=["כב' ה",'כבוד השופט','כבוד השופטת','הרשם','הרשמת']
headWords=["נגד","תובע","נתבע","משיב","מערער",'עו"ד',"פסק דין"]
word2=["לפני :","בפני :","בפני:","בפני:"]

word2.append('בפני')
word2.append('לפני')
shofet2=["כב' ה",'השופט','השופטת','הרשם','הרשמת']
shofet3=["כב' ה",'השופט','הרשם','הרשמת']

In [97]:
import unicodedata
df1 = pd.read_csv('name-gen-tit-engname.csv')
name_full=list()
for i in range(len(df1)):
    p=df1.iloc[i,1]
    name_full.append(p)
reg="\d{2,4}"



srch=list()
wtf=list()
q=-1
for i in name_full:
    q=q+1
    #print (i)
    name=i.replace('ז"ל','')
    #print(name)
    name=name.replace("-"," ")
    name=name.replace('ד"ר','')
    #print(name)
    x=re.search(reg,name)
    if x:
        #print(x)
    
        wtf.append(i)
        
    else:
        #print('else')
        t=list()
        t.append(name)
        #print(t)
        
        t.append(q)
        srch.append(t)
#need to make half name list or other way of srching   #######add only last name
srch_names2=list()
short_names=list()
for i in range(len(srch)):
    juj=list()
    juj.append(srch[i][1])
    juj.append(srch[i][0])
    srch_names2.append(juj)
    
    
    t=srch[i][0].split()
    #print(t)
    if len(t)==2:
        #print("wawaawwawaawawawawawawaw")
        if len(t[1])>4:
            juj=list()
            juj.append(srch[i][1])
        
            juj.append(t[1])
            short_names.append(juj)
            
        
    fname=t[0]
    #srch_trm=''
    srch_trm=fname[:1]+"."
    srch_trm2=fname[:1]+"'"
    lname=''
    
    for j in range(1,len(t)):
        #print('inforloop')
        if len(t[j])>0:    
            lname=lname+str(t[j])+' '
            juj=list()
            juj.append(srch[i][1])
            juj.append(srch_trm+' '+lname)
            srch_names2.append(juj)
            juj=list()
            juj.append(srch[i][1])
            juj.append(srch_trm2+' '+lname)
            srch_names2.append(juj)
#####also need to arrange by len from long to short:

        
srch_names2.append([118, 'אידית קליימן '])


In [98]:
search_names=list()
for i in range(len(srch_names2)):
    search_names.append(srch_names2[i])
for i in range(len(short_names)):
    search_names.append(short_names[i])    

In [99]:
len(search_names)

6542

In [100]:
search_names.append([1374, 'מני מזוז '])

In [101]:
search_names.append([1374, 'מ. מזוז '])

In [102]:
search_names.append([1374, "מ' מזוז "])

In [103]:
search_names[6544][1]

"מ' מזוז "

In [104]:
for i in range(len(search_names)):
    if  search_names[i][1].find('מאזוז')!=-1:
        print(search_names[i])

[1374, 'מנחם (מני) מאזוז ']
[1374, 'מ. (מני) מאזוז ']
[1374, "מ' (מני) מאזוז "]


In [105]:
###HEAD GETTER###   returns precise cutoff
def head_idx(casee):
    #print("start")
    case=casee[:500]
    for word in headWords:
        #print(word)
        t=case.find(word.strip())
        if t!=-1:
            #print('0yay')
            return t
    for word in word2:
        #print(word)
        t=case.find(word.strip())
        if t!=-1:
            #print('01yay')
            return t+100
    for word in shofet3:
        #print(word)
        t=case.find(word.strip())
        if t!=-1:
            #print('02yay')
            return t+100
        
    #if we got here - no matches
    #so we give val
    #print("wtf no matches")
    
    return t

In [107]:
#DiFF:
#get kind of procedure and number for easy search 
regg7 = "[0-9]+"+'\/'+"[0-9]"
regggie4='([0-9])+'+'-'+'([0-9])+'+'-'+'([0-9])+'
regggie1='[א-ת]+'+'"'+'[א-ת]'+' '
regg8=regggie1+regg7
regg9=regggie1+regggie4

def caseNumListBod2(case) :
    #return body cases list
    retList=list()
    y=re.findall(regg8,case)
    #print(y)
    #print(len(y))
    end=0
    for i in range(len(y)):
        x=re.search(regg8,case[end:])
        retList.append(case[x.start()+end:x.end()+end+1])
        end=end+x.end()
    y=re.findall(regg9,case)
    end=0
    #print(y)
    #print(len(y))
    for i in range(len(y)):
        x=re.search(regg9,case[end:])
        retList.append(case[x.start()+end:x.end()+end])
        end=end+x.end()    
    #print(retList)    
    retList=list(dict.fromkeys(retList))    
    
    return retList    

In [108]:
#INPUT: case as txt from case list and headsplitter.
#RETURN: list of pairs -  case num from head and case num from bod.



#caseNumsLists=list()
def nums_returner(case):
    caseNumsList=list()
    #first check bod , then get head , then merge list
    bod = caseNumListBod2(case[1])
    head = caseNumListBod2(case[0])
    #print(bod)
    #print(head)
    if head!=[]:
        
        for i in range(len(bod)):
            lilist=list()
            lilist.append(head[0])
            lilist.append(bod[i])
            caseNumsList.append(lilist)
    
    return caseNumsList

    
    

In [109]:
def ret_bod_names(body):
    lis=list()
    mat=list()
    matches=[(m.start(0)) for m in re.finditer('(השופט)',body)]
    if len(matches)>0:
        for i in matches:
            lis.append(body[i:i+30])
    matches=[(m.start(0)) for m in re.finditer('(הרשם)',body)]
    if len(matches)>0:
        for i in matches:
            lis.append(body[i:i+30]) 
    matches=[(m.start(0)) for m in re.finditer('(הרשמת)',body)]
    if len(matches)>0:
        for i in matches:
            lis.append(body[i:i+30])
    
    return lis
    

In [110]:
def get_edges(head_bod):
    t=ret_head_names(head_bod[0])
    p=ret_bod_names(head_bod[1])
    if len(t)>0:
        if len(p)>0:
            heads=list()
            tails=list()
            for i in range(len(t)):
                k=get_id(t[i])
                if k:
                    heads.append(k)
            for i in range(len(p)):
                k=get_id(p[i])
                if k:
                    tails.append(k)        
            if (len(tails))!=0 and (len(heads))!=0:
            #lists rdy. remove from 2nd the head ids
                hed_id=list(dict.fromkeys(heads))
                tail_id=list()
                for i in range(len(tails)):
                    if tails[i] not in hed_id:
                        tail_id.append(tails[i])
                        
                if (len(tail_id))!=0 and (len(hed_id))!=0:
                    edges=list()
                    for hed in hed_id:
                        for j in range(len(tail_id)):
                            edges.append(str(hed)+';'+str(tail_id[j]))
                    return edges        
    else :
        return None

In [111]:
def get_id(line):
    rets=list()
    line1=line.replace('"','')
    line1=line1.replace('-',' ')
    for name in search_names:
        
        if line1.find(name[1].strip())!=-1:
            rets.append(name[0])
        
    return rets 
            
    

In [112]:
C:/networks-proj/ksharim/crossmatch/TOTNEW.txt
C:\networks-proj\korotHaim

SyntaxError: invalid syntax (1426107879.py, line 1)

In [113]:
df1 = pd.read_csv('C:/networks-proj/korotHaim/id-Byear2.csv')


In [114]:
for i in range(len(df2.index)):
    k=df2.iloc[i,0]
    curr=k.splitlines()
    l_list.append(curr)
print(len(l_list))


NameError: name 'df2' is not defined

In [115]:
id_birth=list()
for i in range(len(df1)):
    line=list()
    line.append(df1.iloc[i,0])
    line.append(df1.iloc[i,1])
    id_birth.append(line)

In [116]:
for i in range(len(id_birth)):
    if id_birth[i][1]>1000:
        id_birth[i][1]=int(id_birth[i][1])

In [118]:
#make get id with file num 
'כבוד השופט נ הנדל, כבוד השופט מ מזוז, כבוד השופטת י וילנר 

SyntaxError: EOL while scanning string literal (1656207486.py, line 2)

In [119]:
t=get_id("כבוד השופט נ' הנדל, כבוד השופט מ' מזוז, כבוד השופטת י' וילנר")

In [120]:
for i in t:
    date=2017
    #if date-id_birth[int(i)][1]>30:
        
    print(i)
    print(date-id_birth[int(i)][1])

306
58
1177
65
306
58
1374
62


In [144]:
filee='C:/networks-proj/ksharim/crossmatch/last/use.txt'

In [145]:
with open(filee, "r",encoding='utf-8') as txt:
        cases=txt.readlines()
        txte=list()
        for i in cases:
            line=i.replace('\n','')
            #line=line.replace("'",'')
        
            if len(line)>1:
                
     
                txte.append(line)

In [146]:
len(txte)

24572

In [147]:
txte[8]

'ע"פ 1530/04;כבוד הרשמת ג\'מילה ג\'בארין כליפה '

In [148]:
casses=list()
for i in range(len(txte)):
    lis=list()
    t=txte[i].split(';')
    tt=t[1].split(",")
    lis.append(t[0].replace('"',''))
    for j in tt:
        lis.append(j)
    casses.append(lis)

In [149]:
len(casses)

24572

In [150]:
len(casses[20811][1])

29

In [151]:
len(casses[4177])

4

In [155]:
fullist=list()
for i in range (len(casses)):
    if len(casses[i][1])<2 and len(casses[i])==2:
        continue
    print(i)
    case=list()
    case.append(casses[i][0])
    space=casses[i][0].find(' ')
    numOnly=casses[i][0][space:]
    casY=numOnly.split('/')
    if cas
    if casY[1]!=-1:
        #we old style
        year=int(casY[1])
    else:
        casY=numOnly.rfind('-')
        year=int(numOnly[casY+1:])
    if year<24:
        year=2000+year
    else:
        year=1900+year
    ####now we have year.
    idlis=list()
    for j in range(1,len(casses[i])-1):
        
        k=get_id(casses[i][j])
        print(k)
        if k!=[]:
            for p in k:
                idlis.append(p)
    idlis=list(dict.fromkeys(idlis)) 
    #print(idlis)
    #now cross year check:
    
    for j in idlis:
        #print("kkk"+str(year-id_birth[int(j)][1]))
        if year-id_birth[int(j)][1]>29:
            #print("kkk"+year-id_birth[int(j)][1])
            case.append(j)
            
    #fullist.append(case) 
    if len(case)>1:
        
        with open('C:/networks-proj/ksharim/crossmatch/caseID73.txt',  'a',encoding='utf-8') as f:
            for i in range(len(case)-1):
                f.write(str(case[i])+';')
            f.write(str(case[len(case)-1])+'\n')   
            
        
        
        
        
        

0
[879]
[]
1


IndexError: list index out of range

In [130]:
def get_ids2(head_bod):
    t=ret_head_names(head_bod[0])
    p=ret_bod_names(head_bod[1])
    if len(t)>0:
        if len(p)>0:
            heads=list()
            tails=list()
            for i in range(len(t)):
                k=get_id(t[i])
                if k:
                    heads.append(k)
            #for i in range(len(p)):
                #k=get_id(p[i])
                #if k:
                 #  tails.append(k)        
            #if (len(tails))!=0 and (len(heads))!=0:
            #lists rdy. remove from 2nd the head ids
            hed_id=list(dict.fromkeys(heads))
                #tail_id=list()
                #for i in range(len(tails)):
                    #if tails[i] not in hed_id:
                        #tail_id.append(tails[i])
                        
                #if (len(tail_id))!=0 and (len(hed_id))!=0:
            edges=list()
            for hed in hed_id:
                        
                edges.append(str(hed))
            return edges        
    else :
        return None

In [131]:
def get_ids(head_bod):
    t=ret_head_names(head_bod[0])
    
    if len(t)>0:
        if 1>0:
            heads=list()
            
            for i in range(len(t)):
                k=get_id(t[i])
                if k:
                    heads.append(k)
            
                hed_id=list(dict.fromkeys(heads))
                
               
                        
                
                return hed_id        
    else :
        return None

In [132]:
#better version:
#we count how many JUJE we have in head if 1 ok else :
regg1='(כבוד)'

def ret_head_names(head):
    liss=list()
    x=re.findall(regg1,head)
    
    if (len(x))>1:
        print('entered many')
        #we found more then 1
        #so we split around them and append them all
        
        y=re.search(regg1,head)
        print(y.start())
        head2=head[y.start():]
        t=head2.split('כבוד')
        for j in range(len(t)):
            if len(t[j])>6:
                if j!=len(t)-1:
                    pos=t[j].find(',')
                    liss.append (t[j][:pos]) 
                else:
                    pos=t[j].find(',')
                    
                    liss.append (t[j][:pos])
    else:
        if len(x)==1:
            x=re.search(regg1,head)
            y=head[x.start()+11:].find(',')
            if y!=-1:
                liss.append(head[x.start():x.start()+y+11])
            else:
                liss.append(head[x.start():x.start()+50])
        else:
            print('wtf')
                    
    return liss    
                
                
                
            
            

In [133]:
## bad cases should go out the loop
bad_cases=list()

good_cases=list()

for case in (cases):
    if head_idx(case)!=-1:
        good_cases.append(case)
    else:
        bad_cases.append(case)

In [134]:
#### NEW
head_bod=list()
for case in good_cases:
    c=list()
    bod_idx=head_idx(case)
    head=case[:bod_idx]
    bod=case[bod_idx:]
    
    c.append(head)
    c.append(bod)
    head_bod.append(c)
    

In [135]:
cut_where=list()
for i in range(len(txte)):
    
    #this is cell so weellll its a stringin each
    kk=txte[i].find("takd_id")
    if kk!=-1:
        #we have match
        cut_where.append(i)
        
cut_where=cut_where[1:]


In [137]:
def year_get(cassse):
    space=cassse.find(' ')
    numOnly=cassse[space:]
    casY=numOnly.rfind('/')
    print(casY)
    if casY!=-1:
        #we old style
        year=int(numOnly[casY+1:])
    else:
        casY=numOnly.rfind('-')
        year=int(numOnly[casY+1:])
    if year<24:
        year=2000+year
    else:
        year=1900+year
    return year    

In [138]:
casses[1][0]

'בתא 47766-03-19'

In [139]:
ppp=caseNumListBod2(casses[1][0])

In [234]:
cases[1]

',takd_id: תא@12949-04-21;takd_s:20230611;takd_d:11.06.2023;takd_u:14.06.2023;mms_s:שלום;mms_p:64495;mms_y:2023;mms_v:2;orig_filename:838147db974343458d6c5964100d096d.html;taktzir_exist:0;kt_exist:0;,בית משפט השלום בנצרת,ת"א 12949-04-21 פלוני נ\' העמק ואח\' ,תיק חיצוני: ,בפני ,כבוד השופטת הבכירה שאדן נאשף-אבו אחמד,תובעת,פלוני,נגד,נתבעים,1. המרכז הרפואי העמק,2. קופת חולים כללית חברה לתועלת הציבור,החלטה,אני קובעת דיון הוכחות ליום 4.2.24 שעה 10:00. ,העדויות הראשיות תישמענה בע"פ, ללא תצהירים. ,התובעת תגיש תיק מוצגים מטעמה תוך 30 יום מהיום ; ,הנתבעים יגישו תיק מוצגים מטעמם תוך 30 יום לאחר מכן. ,הצדדים ידאגו לזמן עדיהם זמן סביר לפני דיון ההוכחות. ,ב"כ הצדדים יהיו ערוכים לסכם טענותיהם בע"פ בתום שמיעת הראיות. ,לדיון מוקצבות שעתיים, כולל זמן סיכומים בע"פ. ,המזכירות תמציא העתק מהחלטה זו לצדדים ותזמנם כדין. ,ניתנה היום, כ"ב סיוון תשפ"ג, 11 יוני 2023, בהעדר הצדדים.'

In [ ]:
###################changesssss
togeph=list()
#bad_cases=list()
count=0
for filename in os.listdir(directory):
    
    if filename.endswith('.txt'):
        count=count+1
        print(count)
        
        
        name='C:/networks-proj/ksharim/readyFull/cases/'+filename
        print(name)
        
        with open(name, "r",encoding='utf-8') as txt:
            mosdot_city=txt.readlines()
        txte=list()
        for i in mosdot_city:
            line=i.replace('\n','')
            if len(line)>1:
                p=re.sub(reg3,'',line)
                p=re.sub(reg4,'',p)
            #p=re.sub(reg5,"'",p)
     
                txte.append(p)
        cut_where=list()
        for i in range(len(txte)):
    
        #this is cell so weellll its a stringin each
            kk=txte[i].find("takd_id")
            if kk!=-1:
            #we have match
                cut_where.append(i)
        
        cut_where=cut_where[1:]
        print("len cutwhere:"+str(len(cut_where)))
        casess=list()
        p=''
        for i in range(len(txte)):
            #print(count)
            p=p+","+(txte[i])
    
            if i+1 in cut_where:
                casess.append(p)
                p=''
                
        #good_cases=list()
        #bad_cases=list()

        ttpp=0
        head_bod=list()
        for case2 in casess:
            ttpp=ttpp+1
            print("cases2 index"+str(ttpp))
            c=list()
            bod_idx=head_idx(case2)
            head=case2[:bod_idx]
            bod=case2[bod_idx:]
    
            c.append(head)
            c.append(bod)
            head_bod.append(c)
            
        countt=0    
        print("len headnod: "+str(len(head_bod)))
        for case in head_bod:
            #countt=countt+1
            #print(countt)
            t=get_id(case[0].replace('-',' '))
            ppp=caseNumListBod2(case[0])
            ttt=caseNumListBod2(case[1])
            
           
                    
                
            
            #v= ret_head_names(case[0])
            #y=get_ids2(case)
            if t!=[] and ppp!=[]:
                q=year_get(ppp[0])
                t=list(dict.fromkeys(t))
                listit=list()
                for tit in t:
                    if year-id_birth[int(tit)][1]>29:
                        listit.append(tit)
                if listit!=[] and len(ttt)>0:        
                    with open('C:/networks-proj/ksharim/crossmatch/casesWeHaveMAX.txt',  'a',encoding='utf-8') as f:
                        f.write(str(ppp[0]) + ';')
                        for lll in range(len(listit)):
                            f.write(str(listit[lll]) + ';')
                        if len(ttt)>0:    
                            for kkk in range(len(ttt)-1):
                                f.write("["+str(ttt[kkk]) +"]"+ ';')
                            
                            f.write("["+str(ttt[len(ttt)-1]) +"]"+ '\n')
                        else:
                            f.write('\n')
                       
           
            
        
            
        
        
        
        
        
        
                                    
#df=pd.DataFrame(togeph)
#name='C:/networks-proj/ksharim/readyFull/caseNums8.csv'
#df.to_csv(name,index=False)

        